In [ ]:
%pip install transformers
%pip install torch
%pip install torchvision


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

In [ ]:
train = pd.read_table('corpus2_IOB.tsv')
dev = pd.read_table('development_IOB.tsv')
evaluate = pd.read_table('evaluation_IOB.tsv')

In [ ]:
def IOBword_to_sent(df,name):
    name=pd.DataFrame()
    data=df.iloc
    sent=""
    labels=""
    patterns=[]
    for i in range(len(data[:,0])-1):
        sent=sent + " " + data[i,0]
        labels=labels + " " + data[i,1]
        if((str(data[i,0]) == ".") & str((data[i+1,0][0])).isupper()):
            if(len(sent) > 512):
                print("wrong length")
            else:
                text_label={'text': sent, 'labels': labels}
                sent=""
                labels=""
                patterns.append(text_label)
    name = pd.DataFrame(patterns)
    return name

df_eval=IOBword_to_sent(evaluate,"df_eval")
df_train=IOBword_to_sent(train,"df_train")
df_dev=IOBword_to_sent(dev,"df_dev")

display(df_train)
# df_train.to_csv("C:/Users/melina/Desktop/EMODnet/NER_BERT/train.tsv",index=False,sep="\t")
# df_eval.to_csv("C:/Users/melina/Desktop/EMODnet/NER_BERT/eval.tsv", index=False,sep="\t")
# df_dev.to_csv("C:/Users/melina/Desktop/EMODnet/NER_BERT/dev.tsv",index=False,sep="\t")


,text,labels
0,The basic procedure in field experiments exam...,O O O O O O O B-LIFE_STAGE I-LIFE_STAGE I-LIF...
1,Although cages are most commonly used to asse...,O O O O O O O O O O O O O O O O O O O O B-DIS...
2,"Atherinids are small marine , estuarine and f...",O O O B-DISTRIBUTION_DESCRIPTOR O B-DISTRIBUT...
3,"In the Southeast Pacific , the family is only...",O O B-DISTRIBUTION_DESCRIPTOR I-DISTRIBUTION_...
4,All freshwater atherinid species are derived ...,O B-DISTRIBUTION_DESCRIPTOR O O O O O B-DISTR...
...,...,...
56,"In Russian waters , Hemilepidotus zapus is di...",O B-DISTRIBUTION_DESCRIPTOR I-DISTRIBUTION_DE...
57,Taking into consideration trawl catches of 19...,O O O O O O O O O O O B-DISTRIBUTION_DESCRIPT...
58,In the northern part of the investigated are...,O O O O O O O O O B-DISTRIBUTION_DESCRIPTOR I...
59,This species is the smallest and the most sho...,O O O O B-BODY_SIZE O O O O O O O O O O O O O...


# Initialize Tokenizer

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Create Dataset Class 

In [ ]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

# Split Data and Define Unique Labels

In [ ]:
a=pd.DataFrame()
labels=""
a['labels']=pd.concat([df_train['labels'],df_eval['labels'],df_dev['labels']])

labels = [i.split() for i in a['labels'].values.tolist()]
unique_labels = set()

for lb in labels:
        [unique_labels.add(i) for i in lb if i not in unique_labels]
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

print(labels_to_ids)
print(ids_to_labels)

# df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
#                             [int(.8 * len(df)), int(.9 * len(df))])
df_train=df_train
df_val=df_dev
df_test=df_eval


{'I-LIFE_STAGE': 0, 'O': 1, 'B-LIFE_STAGE': 2, 'B-DISTRIBUTION_DESCRIPTOR': 3, 'I-BODY_SIZE': 4, 'I-DISTRIBUTION_DESCRIPTOR': 5, 'B-BODY_SIZE': 6}
{0: 'I-LIFE_STAGE', 1: 'O', 2: 'B-LIFE_STAGE', 3: 'B-DISTRIBUTION_DESCRIPTOR', 4: 'I-BODY_SIZE', 5: 'I-DISTRIBUTION_DESCRIPTOR', 6: 'B-BODY_SIZE'}


# Build Model

In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

# Model Training

In [ ]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)
    print(train_dataset)
    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 30
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

100%|██████████| 31/31 [00:06<00:00,  5.14it/s]


Epochs: 1 | Loss:  0.797 | Accuracy:  0.834 | Val_Loss:  0.699 | Accuracy:  0.842


100%|██████████| 31/31 [00:05<00:00,  5.28it/s]


Epochs: 2 | Loss:  0.627 | Accuracy:  0.857 | Val_Loss:  0.660 | Accuracy:  0.842


...

Epochs: 28 | Loss:  0.067 | Accuracy:  0.982 | Val_Loss:  0.899 | Accuracy:  0.866


100%|██████████| 31/31 [00:06<00:00,  5.09it/s]


Epochs: 29 | Loss:  0.071 | Accuracy:  0.982 | Val_Loss:  0.778 | Accuracy:  0.860


100%|██████████| 31/31 [00:06<00:00,  5.08it/s]


Epochs: 30 | Loss:  0.063 | Accuracy:  0.987 | Val_Loss:  0.818 | Accuracy:  0.863


# Evaluate Model

In [ ]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')


evaluate(model, df_test)

Test Accuracy:  0.798


# Predict One Sentence

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

evaluate_one_text(model, 'Newborn young measured from 2 to 20 mm in length')

Newborn young measured from 2 to 20 mm in length
['O', 'O', 'O', 'O', 'I-BODY_SIZE', 'O', 'I-BODY_SIZE', 'O', 'O', 'I-BODY_SIZE']
